In [ ]:
# Install packages, reinstall libraries 
!pip install -q gdown
!pip install --upgrade --quiet \
    numpy==1.26.4 \
    pandas==2.2.2 \
    scikit-learn==1.3.2 \
    xgboost==2.0.3 \
    lightgbm==4.1.0 \
    catboost==1.2.7
!pip install pandas openpyxl
!pip install gridstatus

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
# Import specific modules/functions within packages/libraries

import pandas as pd
from gridstatus import Ercot
from datetime import *
from pathlib import Path

KeyboardInterrupt: 

In [5]:
from pathlib import Path
base_path = Path.home() / "Documents" / "totalenergies_price_forecasting"

In [ ]:
def process_year(year):
    print(f"\n🔄 Processing year: {year}")

    # === Step 1: Load IntGenbyFuel ===
    if year == 2025:
        intgen_file = base_path / "data" / "raw" / "fuel_mix" / "report" / "IntGenbyFuel2025.xlsx"
        months = ['Jan', 'Feb', 'Mar', 'Apr']

    else: 
        intgen_file = base_path / "data" / "raw" / "fuel_mix" / "report" / f"int_gen_by_fuel_{year}.xlsx"
        months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    
    intgen_xls = pd.ExcelFile(intgen_file)
    fuels = ['Coal', 'Biomass', 'Gas', 'Nuclear', "Hydro"]
    all_gen_data = []

    for sheet in months:
        print(f"  • IntGen sheet: {sheet}")
        df = intgen_xls.parse(sheet)
        df = df[df['Fuel'].isin(fuels)].copy()

        time_cols = [col for col in df.columns if ':' in str(col)]
        df_melted = df.melt(id_vars=['Date', 'Fuel'], value_vars=time_cols,
                            var_name='Time', value_name='MW')
        df_melted['Timestamp'] = pd.to_datetime(df_melted['Date'].astype(str) + ' ' + df_melted['Time'], errors='coerce')
        df_melted.dropna(subset=['Timestamp'], inplace=True)
        hourly_df = df_melted.groupby(['Timestamp', 'Fuel'])['MW'].sum().reset_index()
        all_gen_data.append(hourly_df)

    gen_df = pd.concat(all_gen_data, ignore_index=True)

    # Pivot so each fuel becomes its own column
    gen_pivot = gen_df.pivot(index='Timestamp', columns='Fuel', values='MW').reset_index()
    gen_pivot.columns.name = None

    return gen_pivot

Mounted at /content/drive

🔄 Processing year: 2022
  • IntGen sheet: Jan
  • IntGen sheet: Feb
  • IntGen sheet: Mar
  • IntGen sheet: Apr
  • IntGen sheet: May
  • IntGen sheet: Jun
  • IntGen sheet: Jul
  • IntGen sheet: Aug
  • IntGen sheet: Sep
  • IntGen sheet: Oct
  • IntGen sheet: Nov
  • IntGen sheet: Dec
  • RTM sheet: Jan
      Delivery Date  Delivery Hour  Delivery Interval Repeated Hour Flag  \
0        01/01/2022              1                  1                  N   
1        01/01/2022              1                  2                  N   
2        01/01/2022              1                  3                  N   
3        01/01/2022              1                  4                  N   
4        01/01/2022              1                  1                  N   
...             ...            ...                ...                ...   
68443    01/31/2022             24                  2                  N   
68444    01/31/2022             24                  3     

In [ ]:
# === Process years and store final merged DataFrames ===
all_data = []
for year in [2022, 2023, 2024, 2025]:
    df = process_year(year)
    all_data.append(df)

all_data_path = base_path / "data" / "processed" / "Fuel_Mix_All_Years.csv"
combined_all_years = pd.concat(all_data, ignore_index=True)
combined_all_years.to_csv(all_data_path, index=False)
print("📁 Combined all years into 'Fuel_Mix_All_Years.csv'")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>